# Advent of Code 2021
## Day 16
*<https://adventofcode.com/2021/day/16>*

In [1]:
import IPython
import math
import re
from helper import *
from itertools import product, combinations, permutations
from collections import Counter, defaultdict
from rich import inspect, print, pretty
pretty.install()

In [2]:
DAY = 16
inp = get_input(DAY)
part_1 = 0
part_2 = 0

In [3]:
inp_bin = [bin(int(x, 16))[2:].rjust(4, "0") for x in inp]
inp_bin_str = "".join(inp_bin)

In [4]:
class Packet:
    def __init__(self, version: int, id: int):
        self.version = version
        self.id: int = id
        self.data: list[int | Packet] = []
        self.value: int = 0

    def __repr__(self) -> str:
        return f"Packet({self.version}: {self.data})"

    def sum_versions(self) -> int:
        total = self.version
        for x in self.data:
            if isinstance(x, Packet):
                total += x.sum_versions()
        return total

In [5]:
def read_bits(n: int, start: int) -> tuple[int, int]:
    return int(inp_bin_str[start:start+n], 2), start + n

def calc_packet(pac: Packet) -> int:
    if isinstance(pac.data[0], int):
        return pac.data[0]
    
    return {0: sum, 
            1: math.prod, 
            2: min, 
            3: max, 
            5: lambda a: int(a[0] > a[1]), 
            6: lambda a: int(a[0] < a[1]), 
            7: lambda a: int(a[0] == a[1])}[pac.id](tuple(map(calc_packet, pac.data)))

def read_packet(start: int) -> tuple[Packet, int]:
    cur = start
    version, cur = read_bits(3, cur)
    i, cur = read_bits(3, cur)
    pac = Packet(version, i)
    if i == 4:
        num_bin = ""
        cont = True
        while cont:
            val, cur = read_bits(1, cur)
            if val == 0:
                cont = False
            num_bin += inp_bin_str[cur:cur+4]
            cur += 4
        pac.data = [int(num_bin, 2)]
    
    else:
        op_type, cur = read_bits(1, cur)
        if op_type == 0:
            sub_length, cur = read_bits(15, cur)
            start = cur
            while cur < start + sub_length:
                sub, cur = read_packet(cur)
                pac.data.append(sub)
                
        elif op_type == 1:
            num_subs, cur = read_bits(11, cur)
            for _ in range(num_subs):
                sub, cur = read_packet(cur)
                pac.data.append(sub)
    
    pac.value = calc_packet(pac)
    return pac, cur

In [6]:
pac, _ = read_packet(0)
part_1 = pac.sum_versions()
part_2 = pac.value

In [7]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 943

Part 2: 167737115857